In [1]:
import pytz

from metalib.metagomano import MetaGO
from MetaTrader5 import TIMEFRAME_M1
from datetime import datetime, timedelta
import pytz as pytz

In [2]:
end_time = datetime.now(pytz.utc) + timedelta(hours=3)
start_time = end_time - timedelta(days=30)

In [3]:
from MetaTrader5 import copy_rates_range
import MetaTrader5 as mt

In [4]:
import yaml
import warnings
from multiprocessing import Process
from metalib.metaworker import run_strategy_loop
import MetaTrader5 as mt5

warnings.filterwarnings("ignore")

processes = []

with open("../config/dev/metago.yaml", "r") as f:
    config_data = yaml.safe_load(f)

name = "BTCUSD_H1_GO"
entry = config_data[name]


In [5]:
strategy_type = entry.pop("strategy_type")
init_args = entry.copy()

# Convert timeframe string (e.g. "TIMEFRAME_M1") to actual mt5 constant
if isinstance(init_args.get("timeframe"), str):
    init_args["timeframe"] = eval(init_args["timeframe"])

# Convert null active_hours to None
if "active_hours" in init_args and init_args["active_hours"] == "None":
    init_args["active_hours"] = None

In [6]:
strategy_registry = {
    "metago": {
        "class": MetaGO,
        "init_keys": [
            "symbols",
            "timeframe",
            "tag",
            "active_hours",
        ]
    }
}

In [7]:
strategy_registry

{'metago': {'class': metalib.metagomano.MetaGO,
  'init_keys': ['symbols', 'timeframe', 'tag', 'active_hours']}}

In [8]:
init_args

{'symbols': ['BTCUSD'],
 'timeframe': 16385,
 'tag': 'metago_crypto_btc',
 'active_hours': None,
 'risk_factor': 0.09,
 'lookahead': 48}

In [9]:
strategy_info = strategy_registry.get(strategy_type)
if not strategy_info:
    raise ValueError(f"Unknown strategy type: {strategy_type}")

StrategyClass = strategy_info["class"]
expected_keys = strategy_info["init_keys"]

# Ensure required keys are present
for key in expected_keys:
    if key not in init_args:
        raise ValueError(f"Missing required argument: {key}")

instance = StrategyClass(**{k: init_args[k] for k in expected_keys})
instance.connect()
instance.fit()

No model to fit for MetaGO(mano) pelo!


In [10]:
instance.run(start_time, end_time)

True

In [103]:
instance.loadData(start_time, end_time)

In [108]:
instance.sl

118921.83142857144

In [107]:
instance.tp

117898.5

In [105]:
instance.retrieve_indicators(instance.data[init_args["symbols"][0]])

,uptrend,downtrend,sideways,true_open_weekly,true_open_monthly,crossed_diff_monthly_weekly,above_true_open_monthly_diff,above_true_open_weekly_diff,short_sma,long_sma,close,atr
time,,,,,,,,,,,,
2025-07-10 20:00:00,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113666.27,NaN
2025-07-10 21:00:00,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113365.47,NaN
2025-07-10 22:00:00,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113237.97,NaN
2025-07-10 23:00:00,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113456.61,NaN
2025-07-11 00:00:00,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116445.75,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-09 14:00:00,True,False,False,114723.55,117898.5,3174.95,-833.43,2341.52,116757.834167,116657.820417,117065.07,321.000714
2025-08-09 15:00:00,True,False,False,114723.55,117898.5,3174.95,-842.04,2332.91,116800.686667,116683.097500,117056.46,321.593571
2025-08-09 16:00:00,True,False,False,114723.55,117898.5,3174.95,-984.22,2190.73,116833.839167,116685.261250,116914.28,322.465000


In [11]:
instance.run(start_time, end_time)

True